<a href="https://colab.research.google.com/github/yifan-zhou19/iso/blob/master/iso_cg_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
##################
# DATA IMPORTING #
##################

from os.path import exists

!wget -O cwe119_cgd.txt https://raw.githubusercontent.com/CGCL-codes/VulDeePecker/master/CWE-119/CGD/cwe119_cgd.txt

print()

with open("./cwe119_cgd.txt", "r") as cgd:
  print("Source: ",cgd.readline())

--2020-04-15 09:45:23--  https://raw.githubusercontent.com/CGCL-codes/VulDeePecker/master/CWE-119/CGD/cwe119_cgd.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17202057 (16M) [text/plain]
Saving to: ‘cwe119_cgd.txt’

cwe119_cgd.txt      100%[===================>]  16.40M  26.3MB/s    in 0.6s    

2020-04-15 09:45:24 (26.3 MB/s) - ‘cwe119_cgd.txt’ saved [17202057/17202057]


Source:  1 CVE-2010-1444/vlc_media_player_1.1.0_CVE-2010-1444_zipstream.c cfunc 449



In [2]:
import re
re_punctuation_string = '[()\s,/.\']'
programs=[]
program = []
statement=[]
labels = []
vocabulary = [] 
remove_token=['',' ',',','NULL','\n',';']
with open("./cwe119_cgd.txt", "r") as cgd:
  lines = cgd.readlines()
  for line in lines:
    statement=[token for token in re.split(re_punctuation_string,line)if token not in remove_token]
    #print(statement)
    if statement == ['---------------------------------']:
      if program[-1] == ['0']:
        labels.append(0)
      elif program[-1] == ['1']:
        labels.append(1)
      programs.append(program[1:-2])
      for s in program[1:]:
        for t in s:
           if t not in vocabulary:
             vocabulary.append(t)
      #print(program)
      program = []
    else:
      program.append(statement)

print(len(labels))


  
print(len(vocabulary))

39753
22631


In [0]:
# shuffle data
import random
randnum = random.randint(0,100)
random.seed(randnum)
random.shuffle(programs)
random.seed(randnum)
random.shuffle(labels)

In [4]:
training_data = []
for i in range(10000):
  training_data.append((programs[i],labels[i]))
print(training_data[:2])

eval_data=[]
for i in range(10000,15000):
  eval_data.append((programs[i],labels[i]))
print(eval_data[:2])

[([['data', '=', 'NULL;'], ['data', '=', 'new', 'wchar_t[100];'], ['data[0]', '=', 'L', '\\0'], ['wchar_t', '*', 'dataCopy', '=', 'data;'], ['wchar_t', '*', 'data', '=', 'dataCopy;'], ['wchar_t', 'source[100];'], ['wmemset', 'source', 'L', 'C', '100-1'], ['source[100-1]', '=', 'L', '\\0']], 0), ([['data', '=', 'NULL;'], ['if', 'globalTrue'], ['char', '*', 'dataBuffer', '=', 'new', 'char[100];'], ['memset', 'dataBuffer', 'A', '100-1'], ['dataBuffer[100-1]', '=', '\\0'], ['data', '=', 'dataBuffer', '-', '8;'], ['char', 'source[100];'], ['memset', 'source', 'C', '100-1'], ['source[100-1]', '=', '\\0']], 1)]
[([['c', '=', 'split', 'arg[i]', '"="', '&n']], 0), ([['char', 'source[100];']], 0)]


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
#look up table  
#using one-hot embedding
vocabulary_size = len(vocabulary)
def look_up_table(word_idx):
  x = torch.zeros(vocabulary_size).long()
  x[word_idx] = 1.0
  return x
word2idx = {w: idx +1 for (idx, w) in enumerate(vocabulary)}
idx2word = {idx+1: w for (idx, w) in enumerate(vocabulary)}
print(look_up_table(word2idx['1']))

tensor([0, 0, 0,  ..., 0, 0, 0])


In [7]:
context_size = 20
def get_programcontext(program):
  id = 0
  contexts =torch.zeros(context_size,1).long()
  for p in program:
    for t in p:
      if id < context_size:
        contexts[id]= word2idx[t]
        id = id +1
      else:
        return contexts
  return contexts
get_programcontext(programs[4920])

tensor([[   63],
        [13942],
        [13943],
        [13944],
        [    3],
        [ 1923],
        [    5],
        [  285],
        [    8],
        [ 1015],
        [    5],
        [  364],
        [11266],
        [  794],
        [  470],
        [13924],
        [  108],
        [    8],
        [13945],
        [    5]])

In [16]:


torch.manual_seed(1)
embedding_dim =15
hidden_dim=15
class ProgrammingML(nn.Module):
  def __init__(self, vocab_size, embedding_dim):
    super(ProgrammingML, self).__init__()
    self.embeddings = nn.Embedding(vocab_size, embedding_dim)
    self.lstm = nn.LSTM(embedding_dim, hidden_dim)
    self.fc1 = nn.Linear(hidden_dim, 2)

  def forward(self, inputs):
    #print(inputs.size())
    embeds = self.embeddings(inputs)
    #print(embeds.size())
    lstm_out, _ = self.lstm(embeds.view(len(inputs), 1, -1))
    #print(out.size())
    out =  self.fc1(lstm_out.view(len(inputs), -1))
    #print(out.size())
    out = F.softmax(out,dim=1)
    return out[-1]


losses = []
loss_function = nn.BCELoss()
model = ProgrammingML(vocabulary_size, embedding_dim)
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(20):
  losses = 0
  for program, label in training_data:
   # print(program)
   # print(label)
   if label ==1:
     out_labels = torch.FloatTensor([0,1])
   else:
     out_labels  = torch.FloatTensor([1,0])

   context_idx = get_programcontext(program)
   model.zero_grad()
   out = model(context_idx)
   #print(out.size())
   #print(f'Out {out}')
   #print(out_labels)
   loss = loss_function(out, out_labels )
   #print(f'Loss {loss.item()}')
   loss.backward()
   optimizer.step()
   #print(f'Loss {loss.item()}')
   losses +=loss.item()
  print(f'Loss at epo {epoch}: {losses/len(training_data)}')

Loss at epo 0: 0.42527033230392264
Loss at epo 1: 0.2945441125874175
Loss at epo 2: 0.2568398194457404
Loss at epo 3: 0.23589682400966994
Loss at epo 4: 0.2215443691034423
Loss at epo 5: 0.20977593923241292
Loss at epo 6: 0.20020808966690384
Loss at epo 7: 0.19178073123257708
Loss at epo 8: 0.18497784415003662
Loss at epo 9: 0.17921896629857073
Loss at epo 10: 0.17528437297012023
Loss at epo 11: 0.17143849855274493
Loss at epo 12: 0.16783472007956007
Loss at epo 13: 0.16382886878197989
Loss at epo 14: 0.16135633647054984
Loss at epo 15: 0.15746435107336693
Loss at epo 16: 0.15545464965120126
Loss at epo 17: 0.15383958924473362
Loss at epo 18: 0.1518017365943333
Loss at epo 19: 0.149097337602331


In [17]:
with torch.no_grad():
  accurate = 0
  for program, label in eval_data:
   # print(program)
   # print(label)
   context_idx = get_programcontext(program)
   model.zero_grad()
   out = model(context_idx)
   #print(f'{out}:{label}')
   if(out[0]>0.5):
     if(label == 0):
       accurate =accurate+1
   elif(out[1]>0.5):
     if(label == 1):
       accurate = accurate+1
  print(f'Accuracy: {accurate/len(eval_data)}')   

Accuracy: 0.8652
